<a href="https://colab.research.google.com/github/seungmanchoi/category_classification/blob/main/fasttext%26LSTM_category_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296185 sha256=aed64460bbde1818535266cfc90ab36b743def9c5322a9bbc797ac215fa9884b
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
import fasttext
import pandas as pd
import re
import sys
import os
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers import normalizers
from tokenizers.normalizers import NFKC
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

In [ ]:
# 설정
epoch_count = 10
word_ngrams = 3
learning_rate = 0.1

# 학습 데이터 경로
train_data = '/content/drive/MyDrive/Colab Notebooks/item_category_classification/product_category_from_fullset_train_cleaned_filtered_tokenized.tsv'

# tqdm을 사용한 학습 진행 표시
for epoch in tqdm(range(1, epoch_count + 1)):
    # 각 epoch에서 모델을 학습
    model = fasttext.train_supervised(train_data, wordNgrams=word_ngrams, epoch=epoch, lr=learning_rate)

100%|██████████| 10/10 [47:28<00:00, 284.81s/it]


In [ ]:
# 마지막 모델 저장 (필요할 경우)
model.save_model('/content/drive/MyDrive/Colab Notebooks/item_category_classification/tokenized_product_category_from_fullset_model_epoch10.bin')

In [ ]:
# 1. BPE 토크나이저 생성
tokenizer = Tokenizer(BPE())
tokenizer.normalizer = normalizers.Sequence([NFKC()])
tokenizer.pre_tokenizer = Whitespace()

# 2. BPE 트레이너 설정
trainer = BpeTrainer(special_tokens=["<unk>"], vocab_size=30000)

# 3. 학습 데이터로 BPE 학습
path = '/content/drive/MyDrive/Colab Notebooks/item_category_classification/'
files = [os.path.join(path, file_name) for file_name in os.listdir(path) if file_name.endswith('.tsv')]

tokenizer.train(files, trainer)

In [ ]:
def bpe_tokenize(text):
    # BPE 토크나이징 수행
    output = tokenizer.encode(text)
    # 공백으로 구분된 토큰을 반환
    return ' '.join(output.tokens)

In [ ]:
def print_predictions():
    print('\n분류할 상품명을 입력하세요 (종료하려면 Ctrl+Z 입력): ', file=sys.stderr)
    while True:
        try:
            query = input().rstrip()
            if not query:
                break  # Exit if input is empty

            tokenized_query = bpe_tokenize(query)
            print(f'Tokenized query: {tokenized_query}')

            result = model.predict(tokenized_query)

            if result:
                print(result)
                print()
            else:
                print('\n결과가 없습니다.')

        except EOFError:
            # Ctrl+Z가 입력되면 EOFError 발생
            print("\n입력이 종료되었습니다.")
            break

        except Exception as e:
            print(f"Error occurred: {e}")
            break

In [ ]:
print_predictions()


분류할 상품명을 입력하세요 (종료하려면 Ctrl+Z 입력): 


앙고라오버핏가디건
Tokenized query: 앙고라 오버핏 가디건
(('__label__의류_여성아우터_가디건',), array([0.63917112]))

오 대박
Tokenized query: 오 대박
(('__label__아동복_상의_티셔츠',), array([0.45340559]))

이제 좀 잘 나오는구만
Tokenized query: 이 제 좀 잘 나오는 구 만
(('__label__악세사리_귀걸이_스터드',), array([0.03288327]))

누나가만들어준예쁜귀걸이
Tokenized query: 누 나가 만들 어 준 예쁜 귀걸이
(('__label__악세사리_귀걸이_기타',), array([0.6898616]))

내가만든은침피어싱
Tokenized query: 내가 만 든 은침 피어싱
(('__label__악세사리_피어싱_써지컬',), array([0.48618466]))




In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/item_category_classification/product_category_from_fullset_test_cleaned_filtered_tokenized.tsv", sep ="\t")

test_df.columns = ['category', 'product_name']

def clean_label(label):
    # 정규 표현식을 사용해 문자, 숫자, _만 남김
    return re.sub(r"^\('([^']*)',\)$", r'\1', label)

predictions = []
for line in test_df['product_name']:
    pred_label = model.predict(line)[0]  # 튜플의 첫 번째 요소 가져오기
    pred_label_str = str(pred_label)
    clean_pred_label = clean_label(pred_label_str)  # 특수문자 제거
    predictions.append(clean_pred_label)

# 새로운 열에 예측값 추가
test_df['predicted_label'] = predictions

# 실제 라벨과 예측 라벨을 _로 분리하여 다중 라벨로 변환
mlb = MultiLabelBinarizer()
actual_labels_binary = mlb.fit_transform([label.split('_') for label in test_df['category']])
predictions_binary = mlb.transform([label.split('_') for label in test_df['predicted_label']])

# 정확도 측정
accuracy = accuracy_score(actual_labels_binary, predictions_binary)

# Precision, Recall, F1-score 계산
precision = precision_score(actual_labels_binary, predictions_binary, average='micro')
recall = recall_score(actual_labels_binary, predictions_binary, average='micro')
f1 = f1_score(actual_labels_binary, predictions_binary, average='micro')

print("======================================")
print(f'Accuracy: {accuracy:.4f}')
print(f'Overall Precision: {precision:.4f}')
print(f'Overall Recall: {recall:.4f}')
print(f'Overall F1-score: {f1:.4f}')
print("======================================")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['주방', '팔찌/목걸이'] will be ignored
  warnings.warn(


Accuracy: 0.7814
Overall Precision: 0.9176
Overall Recall: 0.9172
Overall F1-score: 0.9174


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
# 2단계: 문장 임베딩을 위한 함수 정의
def get_sentence_embedding(sentence):
    words = sentence.split()
    embeddings = [model.get_word_vector(word) for word in words]
    return np.mean(embeddings, axis=0)

# LSTM 모델 정의
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # 마지막 타임 스탭의 출력 사용
        return out

In [ ]:
# 3단계: 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, sentences, labels):
        self.sentences = sentences
        self.labels = labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx], self.labels[idx]

In [ ]:
# 데이터 로드 및 처리
def load_data(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            label, text = line.split("\t", 1)  # Assuming the format is __label__category text
            sentences.append(text.strip())
            labels.append(label)
    return sentences, labels

In [ ]:
# 데이터 로드
train_sentences, train_labels = load_data('/content/drive/MyDrive/Colab Notebooks/item_category_classification/product_category_from_fullset_train_cleaned_filtered_tokenized.tsv')
test_sentences, test_labels = load_data('/content/drive/MyDrive/Colab Notebooks/item_category_classification/product_category_from_fullset_test_cleaned_filtered_tokenized.tsv')

In [ ]:
# 문장 임베딩 생성
train_embeddings = np.array([get_sentence_embedding(sentence) for sentence in train_sentences])
test_embeddings = np.array([get_sentence_embedding(sentence) for sentence in test_sentences])

In [ ]:
# 라벨을 정수로 변환
label_to_index = {label: idx for idx, label in enumerate(set(train_labels))}

# test_labels에서 train_labels에 없는 라벨 추가
for label in set(test_labels):
    if label not in label_to_index:
        # 새로운 라벨에 대한 인덱스 추가
        label_to_index[label] = len(label_to_index)

# 인덱싱
train_labels_indexed = [label_to_index[label] for label in train_labels]
test_labels_indexed = [label_to_index[label] for label in test_labels]


In [ ]:
# LSTM 하이퍼파라미터 설정
input_size = train_embeddings.shape[1]  # fastText 임베딩 차원
hidden_size = 128
output_size = len(label_to_index)  # 클래스 수
batch_size = 32

In [ ]:
# LSTM 모델 초기화
lstm_model = LSTMClassifier(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)


In [ ]:
# 데이터셋과 데이터로더 생성
train_dataset = TextDataset(train_embeddings, train_labels_indexed)
test_dataset = TextDataset(test_embeddings, test_labels_indexed)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 4단계: LSTM 모델 학습
lstm_model.train()
for epoch in range(10):  # 예시로 10 에폭
    total_loss = 0
    for sentences, labels in tqdm(train_loader):
        # 텐서로 변환
        sentences_tensor = torch.tensor(sentences, dtype=torch.float32).unsqueeze(1)  # (batch_size, sequence_length, input_size)
        labels_tensor = torch.tensor(labels, dtype=torch.long)

        optimizer.zero_grad()
        outputs = lstm_model(sentences_tensor)  # LSTM 모델에 입력
        loss = criterion(outputs, labels_tensor)  # 손실 계산
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch + 1}/10], Loss: {total_loss / len(train_loader):.4f}')

print("Training complete.")

  0%|          | 0/10788 [00:00<?, ?it/s]<ipython-input-55-f9b393df14c8>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentences_tensor = torch.tensor(sentences, dtype=torch.float32).unsqueeze(1)  # (batch_size, sequence_length, input_size)
<ipython-input-55-f9b393df14c8>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(labels, dtype=torch.long)
100%|██████████| 10788/10788 [00:43<00:00, 247.25it/s]


Epoch [1/10], Loss: 1.0313


100%|██████████| 10788/10788 [00:44<00:00, 243.43it/s]


Epoch [2/10], Loss: 0.7416


100%|██████████| 10788/10788 [00:42<00:00, 252.08it/s]


...

Epoch [8/10], Loss: 0.6192


100%|██████████| 10788/10788 [00:43<00:00, 248.14it/s]


Epoch [9/10], Loss: 0.6102


100%|██████████| 10788/10788 [00:42<00:00, 254.70it/s]

Epoch [10/10], Loss: 0.6020
Training complete.


In [ ]:
# 5단계: 모델 평가
def evaluate_model(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for sentences, labels in data_loader:
            sentences_tensor = torch.tensor(sentences, dtype=torch.float32).unsqueeze(1)
            labels_tensor = torch.tensor(labels, dtype=torch.long)

            outputs = model(sentences_tensor)
            _, predicted = torch.max(outputs, 1)
            total += labels_tensor.size(0)
            correct += (predicted == labels_tensor).sum().item()

            all_labels.extend(labels_tensor.numpy())
            all_predictions.extend(predicted.numpy())

    accuracy = correct / total
    return accuracy, all_labels, all_predictions

# 평가
accuracy, true_labels, predicted_labels = evaluate_model(lstm_model, test_loader)
print(f'Accuracy: {accuracy:.4f}')

<ipython-input-56-a30b29ca1ded>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentences_tensor = torch.tensor(sentences, dtype=torch.float32).unsqueeze(1)
<ipython-input-56-a30b29ca1ded>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(labels, dtype=torch.long)


Accuracy: 0.7654


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers import normalizers
from tokenizers.normalizers import NFKC

In [ ]:
# 1. BPE 토크나이저 생성
tokenizer = Tokenizer(BPE())
tokenizer.normalizer = normalizers.Sequence([NFKC()])
tokenizer.pre_tokenizer = Whitespace()

# 2. BPE 트레이너 설정
trainer = BpeTrainer(special_tokens=["<unk>"], vocab_size=30000)

# 3. 학습 데이터로 BPE 학습
file = '/content/drive/MyDrive/Colab Notebooks/item_category_classification/complete_filtered.txt'  # 학습 데이터 파일 경로 리스트
tokenizer.train([file], trainer)

# 4. 토큰화 함수 정의
def bpe_tokenize(text):
    # BPE 토크나이징 수행
    output = tokenizer.encode(text)
    # 공백으로 구분된 토큰을 반환
    return ' '.join(output.tokens)

In [ ]:
import torch.nn.functional as F  # Import the functional API for softmax

# 6단계: 입력 문자열로 라벨 예측 및 확률 반환
def predict(input_string, model, tokenizer, label_to_index):
    # BPE 토크나이징
    tokens = bpe_tokenize(input_string)

    # 문장 임베딩 생성
    input_embedding = get_sentence_embedding(tokens)

    # 텐서로 변환
    input_tensor = torch.tensor(input_embedding, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # (1, 1, input_size)

    # 모델 예측
    model.eval()  # 평가 모드
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output, dim=1)  # 소프트맥스 적용
        predicted_label_index = torch.max(probabilities, 1)[1]

    # 인덱스를 라벨로 변환
    predicted_label = [key for key, value in label_to_index.items() if value == predicted_label_index.item()]
    predicted_probability = probabilities[0][predicted_label_index.item()].item()  # 예측 확률

    return predicted_label[0].replace("__label__", "") if predicted_label else None, predicted_probability  # 예측된 라벨과 확률 반환


In [ ]:
# 예시: 입력 문자열로 예측 수행
input_string = "앙고라오버핏가디건"  # 입력 문자열을 여기에 작성
predicted_label, predicted_probability = predict(input_string, lstm_model, tokenizer, label_to_index)
print(f'Predicted Label: {predicted_label}, Probability: {predicted_probability:.4f}')

Predicted Label: 의류_여성아우터_가디건, Probability: 0.4975
